# **Info**

* https://www.youtube.com/watch?v=HbyN_ey-JVc&list=PLB9cczsBXUdmc7owi1GcGJoOc5c4vBPFh&index=10
* kaggle datasets download -d debasisdotcom/parkinson-disease-detection

# **Setup**

In [1]:
!python3 --version

from psutil import *
print(f'CPU Core Count: {cpu_count()}')
print(f'System RAM: {virtual_memory()[0]//(1024**3)} GB')

Python 3.11.4


CPU Core Count: 8
System RAM: 8 GB


In [2]:
import os
from IPython.display import clear_output
from zipfile import ZipFile

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px 
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# **Data Import**

In [3]:
! kaggle datasets download -d debasisdotcom/parkinson-disease-detection

parkinson-disease-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# Read Zip File
Zip_File = ZipFile('parkinson-disease-detection.zip')

print('No. Of File Found: ',len(Zip_File.namelist()))
print('Files Found: ','\n',Zip_File.namelist())

# Extract Files
for file in Zip_File.namelist():
    file_content = Zip_File.read(file)
    with open(os.path.basename(file), "wb") as f:
                f.write(file_content)

No. Of File Found:  1
Files Found:  
 ['Parkinsson disease.csv']


In [5]:
df = pd.read_csv('Parkinsson disease.csv')

print(df.shape)
df.head()

(195, 24)


,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


# **Data Exploration**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [7]:
df.isna().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [8]:
df.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,...,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,154.228641,197.104918,116.324631,0.006220,0.000044,0.003306,0.003446,0.009920,0.029709,0.282251,...,0.046993,0.024847,21.885974,0.753846,0.498536,0.718099,-5.684397,0.226510,2.381826,0.206552
std,41.390065,91.491548,43.521413,0.004848,0.000035,0.002968,0.002759,0.008903,0.018857,0.194877,...,0.030459,0.040418,4.425764,0.431878,0.103942,0.055336,1.090208,0.083406,0.382799,0.090119
min,88.333000,102.145000,65.476000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,...,0.013640,0.000650,8.441000,0.000000,0.256570,0.574282,-7.964984,0.006274,1.423287,0.044539
25%,117.572000,134.862500,84.291000,0.003460,0.000020,0.001660,0.001860,0.004985,0.016505,0.148500,...,0.024735,0.005925,19.198000,1.000000,0.421306,0.674758,-6.450096,0.174351,2.099125,0.137451
50%,148.790000,175.829000,104.315000,0.004940,0.000030,0.002500,0.002690,0.007490,0.022970,0.221000,...,0.038360,0.011660,22.085000,1.000000,0.495954,0.722254,-5.720868,0.218885,2.361532,0.194052
75%,182.769000,224.205500,140.018500,0.007365,0.000060,0.003835,0.003955,0.011505,0.037885,0.350000,...,0.060795,0.025640,25.075500,1.000000,0.587562,0.761881,-5.046192,0.279234,2.636456,0.252980
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,...,0.169420,0.314820,33.047000,1.000000,0.685151,0.825288,-2.434031,0.450493,3.671155,0.527367


In [9]:
# Value Count Of Target Variable
df['status'].value_counts()

1    147
0     48
Name: status, dtype: int64

In [10]:
px.bar(df['status'].value_counts(),title='Count Of Status',width=800,height=500).update_traces(texttemplate='%{y:.2s}')

In [11]:
# Groupby Patient Status
df.groupby('status').mean()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
status,,,,,,,,,,,,,,,,,,,,,
0,181.937771,223.636750,145.207292,0.003866,0.000023,0.001925,0.002056,0.005776,0.017615,0.162958,...,0.013305,0.028511,0.011483,24.678750,0.442552,0.695716,-6.759264,0.160292,2.154491,0.123017
1,145.180762,188.441463,106.893558,0.006989,0.000051,0.003757,0.003900,0.011273,0.033658,0.321204,...,0.027600,0.053027,0.029211,20.974048,0.516816,0.725408,-5.333420,0.248133,2.456058,0.233828


# **Data Pre-Processing**

## **Data Split - Predictors & Target**

In [12]:
df.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [13]:
x = df.drop(['name','status'],axis=1)
y = df['status']

In [14]:
x.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [15]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: status, dtype: int64

## **Data Split - Train & Test**

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [18]:
print(x.shape,x_train.shape,x_test.shape)

(195, 22) (156, 22) (39, 22)


## **Data Standardization**

In [19]:
Feature_Transformer_01 = StandardScaler()
Feature_Transformer_01.fit(x_train)

StandardScaler()

In [20]:
x_train = Feature_Transformer_01.transform(x_train)
x_test = Feature_Transformer_01.transform(x_test)

In [22]:
x_train[0:2]

array([[ 0.63239631, -0.02731081, -0.87985049,  0.36806203,  0.14406765,
         0.49280653,  0.30476869,  0.49170115, -0.3137565 , -0.32592164,
        -0.25848708, -0.23791175, -0.33873289, -0.25879402, -0.2113724 ,
        -0.61257363,  0.87112118,  0.24786644,  0.19888239, -0.97586547,
        -0.55160318,  0.07769494],
       [-1.05512719, -0.83337041, -0.9284778 , -0.12997764,  0.14406765,
        -0.15236125, -0.14436412, -0.15344933,  0.44321834,  0.51889992,
        -0.06490951,  0.09074355,  0.86575291, -0.06521554, -0.15277503,
        -0.2558165 ,  1.46329474, -0.46857462,  0.42875723,  0.3981808 ,
        -0.61014073,  0.39291782]])

# **Model Implementation**

In [23]:
model_01 = svm.SVC(kernel='linear')

# Training
model_01.fit(x_train,y_train)

SVC(kernel='linear')

In [24]:
# Training Data Accuracy
y_pred_train = model_01.predict(x_train)
print(accuracy_score(y_train,y_pred_train))

0.8846153846153846


In [25]:
# Training Data Accuracy
y_pred = model_01.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.8717948717948718


# **Deployment**

In [41]:
input_data = (156.405,189.398,142.822,0.00768,0.00005,0.00372,0.00399,0.01116,0.03995,0.348,0.01721,0.02374,
              0.0431,0.05164,0.03365,17.153,0.649554,0.68608,-4.554466,0.340176,2.856676,0.322111)

input_data_01 = np.asarray(input_data)
print('Input Data 01 Shape: ',input_data_01.shape)

input_data_02 = input_data_01.reshape(1,-1)
print('Input Data 02 Shape: ',input_data_02.shape)

input_data_03 = Feature_Transformer_01.transform(input_data_02)

Prediction_Results = model_01.predict(input_data_03)
print('Prediction Result: ',Prediction_Results,'\n')

if Prediction_Results[0] == 0:
    print('Patient Does Not Have Parkinson')
else:
    print('Patient Has Parkinsons Disease!')

Input Data 01 Shape:  (22,)
Input Data 02 Shape:  (1, 22)
Prediction Result:  [1] 

Patient Has Parkinsons Disease!
